In [1]:
!gcloud dataproc clusters list --region us-central1 

ERROR: (gcloud.dataproc.clusters.list) The required property [project] is not currently set.
It can be set on a per-command basis by re-running your command with the [--project] flag.

You may set it for your current workspace by running:

  $ gcloud config set project VALUE

or it can be set temporarily by the environment variable [CLOUDSDK_CORE_PROJECT]


In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
bucket_name = '207044777' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd

In [ ]:
parquetFile.count()

In [ ]:
%cd -q /home/dataproc
!ls inverted_index_gcp_new.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp_new.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp_new import InvertedIndex

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

In [1]:
def word_count(text, id):
  anchor_links = dict()
  for tup in text:
    tokens = [token.group() for token in RE_WORD.finditer(tup[1].lower())]
    tokens = [token for token  in tokens if token not in all_stopwords]
    for token in tokens:
      if token in anchor_links.keys():
        anchor_links[token].add(tup[0])
      else:
        anchor_links[token] = set([tup[0]])

  res = []
  for key, value in anchor_links.items():
     for doc_id in value:
      res.append((key, (id, doc_id)))

  return res

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl,key=itemgetter(0))

def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))

def partition_postings_and_write(postings):
  return postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey().map(lambda x : InvertedIndex.write_a_posting_list(x,bucket_name))

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings).collect()
index_const_time = time() - t_start

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
bucket_name = 'pkl_207044777'
# Create inverted index instance
inverted = InvertedIndex('anchor_207044777')
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst